<a href="https://colab.research.google.com/github/rahakaushik/quant_finance/blob/main/Implied_Volatility_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Libraries

In [ ]:
!pip install py_vollib

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for py_vollib: filename=py_vollib-1.0.1-py3-none-any.whl size=62832 sha256=7e80e43e35a1f22db4b8b6d8256cc5eb0bb79c849b37352efd02b5210674ab3c
  Stored in directory: /root/.cache/pip/wheels/c0/5c/8f/1ed590a10a2cca3cbfa7a7caa29fb5d729b3e1f819bfda4b5e
  Created wheel for py_lets_be_rational: filename=py_lets_be_rational-1.0.1-py3-none-any.whl size=24450 sha256=15ab9529aa9af4332ff7c50e3c72b114183470dc6d9809e5dc4ae2b202b7cc44
  Stored in directory: /root/.cache/pip/wheels/d9/20/b1/018f644bacb669d5cd6af600fe09ee0729ae5d75875a9f3043
Successfully built py_vollib py_lets_be_rational


### 1. Get Options Data

In [ ]:
import yfinance as yf  # Import yfinance for financial data
import pandas as pd  # Import pandas for data manipulation

ticker = "GOOG"  # Example ticker

def get_options_data(ticker):
    asset = yf.Ticker(ticker)  # Create a Ticker object for the specified ticker
    exp_dates = asset.options  # Get available expiration dates for options
    exp_dates = exp_dates[:8]

    if not exp_dates:  # Check if there are no expiration dates available
        print(f"No options data available for ticker {ticker}.")
        return None, None

    recent_price = asset.history(period="1d")["Close"].iloc[-1]  # Get the most recent closing price

    options_data = []  # Initialize an empty list to hold options data
    for date in exp_dates:  # Loop through each expiration date
        calls = asset.option_chain(date).calls  # Get call options data for the date
        puts = asset.option_chain(date).puts  # Get put options data for the date
        calls["expiration"] = date  # Add expiration date to calls data
        puts["expiration"] = date  # Add expiration date to puts data
        calls["type"] = "call"  # Add option type to calls data
        puts["type"] = "put"  # Add option type to puts data
        data = pd.concat([calls, puts])  # Concatenate calls and puts data
        options_data.append(data)  # Append the data to the options_data list

    if not options_data:  # Check if there is no valid options data
        print(f"No valid options data available for ticker {ticker}.")
        return None, None

    options_data = pd.concat(options_data)  # Concatenate all options data into a single DataFrame
    options_data = options_data[options_data["strike"].between(recent_price * 0.9, recent_price * 1.1)]  # Filter data to include strikes within 10% of the recent price
    options_data["implied_volatility"] = options_data["impliedVolatility"] * 100  # Convert implied volatility to percentage

    return options_data, recent_price  # Return the options data and recent price

options_data, recent_price = get_options_data(ticker)  # Get options data for the specified ticker
options_data  # Display the options data


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expiration,type,implied_volatility
8,GOOG250314C00152500,2025-03-14 19:54:10+00:00,152.5,15.15,14.20,16.10,-1.550001,-9.281444,8.0,27,0.804689,True,REGULAR,USD,2025-03-14,call,80.468945
9,GOOG250314C00155000,2025-03-14 16:58:11+00:00,155.0,12.40,11.90,13.70,2.650000,27.179483,3.0,168,0.931641,True,REGULAR,USD,2025-03-14,call,93.164131
10,GOOG250314C00157500,2025-03-14 18:53:29+00:00,157.5,9.65,9.20,10.55,1.680000,21.079044,60.0,206,0.964844,True,REGULAR,USD,2025-03-14,call,96.484410
11,GOOG250314C00160000,2025-03-14 19:31:57+00:00,160.0,8.05,7.15,8.75,3.010000,74.504960,83.0,237,0.724612,True,REGULAR,USD,2025-03-14,call,72.461213
12,GOOG250314C00162500,2025-03-14 19:54:10+00:00,162.5,5.15,4.85,5.40,2.340000,83.274025,170.0,329,0.517583,True,REGULAR,USD,2025-03-14,call,51.758295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,GOOG250516P00160000,2025-03-14 19:49:47+00:00,160.0,5.90,5.75,5.85,-1.550000,-20.805367,79.0,4040,0.335090,False,REGULAR,USD,2025-05-16,put,33.508966
16,GOOG250516P00165000,2025-03-14 19:29:39+00:00,165.0,7.99,7.65,7.80,-0.830000,-9.410430,624.0,3547,0.326057,False,REGULAR,USD,2025-05-16,put,32.605654
17,GOOG250516P00170000,2025-03-14 19:39:50+00:00,170.0,10.25,9.95,10.10,-0.970000,-8.645279,45.0,3035,0.314460,True,REGULAR,USD,2025-05-16,put,31.445998
18,GOOG250516P00175000,2025-03-14 19:51:50+00:00,175.0,12.40,12.65,12.85,-2.530001,-16.546766,37.0,2952,0.303108,True,REGULAR,USD,2025-05-16,put,30.310755


## 2. Volatility Smile Plot

This block of code creates a function to plot the volatility smile for a given ticker's options data. It first filters the options data into calls and puts, then iterates over each expiration date to plot implied volatility against strike prices. It uses subplots to separately display the volatility smiles for calls and puts, and includes average implied volatility lines for each. The function also calculates overall average implied volatility for both calls and puts, adding them as horizontal lines in the plot. Finally, the function returns these averages and other relevant data for further analysis.

In [ ]:
import plotly.graph_objects as go  # Import Plotly for graph objects
import plotly.express as px  # Import Plotly Express for easy plotting
from plotly.subplots import make_subplots  # Import Plotly for creating subplots

def plot_volatility_smile(options_data, recent_price, ticker):
    if options_data is not None:  # Check if options data is available
        calls_data = options_data[options_data["type"] == "call"]  # Filter for call options
        puts_data = options_data[options_data["type"] == "put"]  # Filter for put options

        expirations = options_data['expiration'].unique()  # Get unique expiration dates
        color_map_2d = px.colors.qualitative.Prism  # Define a color map for plotting

        fig = make_subplots(rows=1, cols=2, subplot_titles=["Calls", "Puts"], shared_yaxes=True)  # Create subplots for calls and puts

        for exp, color in zip(expirations, color_map_2d):  # Loop through expiration dates and colors
            exp_calls = calls_data[calls_data["expiration"] == exp]  # Filter call options by expiration date
            exp_puts = puts_data[puts_data["expiration"] == exp]  # Filter put options by expiration date

            fig.add_trace(go.Scatter(x=exp_calls["strike"], y=exp_calls["implied_volatility"], mode='markers',
                                     marker=dict(color=color), name=exp), row=1, col=1)  # Plot call options
            fig.add_trace(go.Scatter(x=exp_puts["strike"], y=exp_puts["implied_volatility"], mode='markers',
                                     marker=dict(color=color), name=exp, showlegend=False), row=1, col=2)  # Plot put options

        avg_iv_by_strike_calls = calls_data.groupby("strike")["implied_volatility"].mean()  # Calculate average IV by strike for calls
        avg_iv_by_strike_puts = puts_data.groupby("strike")["implied_volatility"].mean()  # Calculate average IV by strike for puts

        fig.add_trace(go.Scatter(x=avg_iv_by_strike_calls.index, y=avg_iv_by_strike_calls.values, mode='lines',
                                 line=dict(color='black', dash='dash'), name='Avg. IV by Strike (Calls)'), row=1, col=1)  # Plot average IV for calls
        fig.add_trace(go.Scatter(x=avg_iv_by_strike_puts.index, y=avg_iv_by_strike_puts.values, mode='lines',
                                 line=dict(color='black', dash='dash'), name='Avg. IV by Strike (Puts)', showlegend=False), row=1, col=2)  # Plot average IV for puts

        overall_avg_iv_calls = calls_data["implied_volatility"].mean()  # Calculate overall average IV for calls
        overall_avg_iv_puts = puts_data["implied_volatility"].mean()  # Calculate overall average IV for puts

        fig.add_hline(y=overall_avg_iv_calls, line=dict(color='gray', dash='dash'),
                      annotation_text=f"Overall Avg. IV (Calls): {overall_avg_iv_calls:.2f}%",
                      row=1, col=1)  # Add horizontal line for overall average IV of calls
        fig.add_hline(y=overall_avg_iv_puts, line=dict(color='gray', dash='dash'),
                      annotation_text=f"Overall Avg. IV (Puts): {overall_avg_iv_puts:.2f}%",
                      row=1, col=2)  # Add horizontal line for overall average IV of puts

        fig.update_layout(title=f"{ticker} Volatility Smile - Current Price: {recent_price:.2f}", showlegend=True, legend_title_text='Expiration Date')  # Update layout
        fig.update_xaxes(title_text="Strike Price")  # Set x-axis title
        fig.update_yaxes(title_text="Implied Volatility (%)")  # Set y-axis title

        fig.show()  # Display the plot

        # Return necessary data for interpretation
        return overall_avg_iv_calls, overall_avg_iv_puts, avg_iv_by_strike_calls, avg_iv_by_strike_puts, expirations

# Assuming options_data and recent_price are already defined
overall_avg_iv_calls, overall_avg_iv_puts, avg_iv_by_strike_calls, avg_iv_by_strike_puts, expirations = plot_volatility_smile(options_data, recent_price, ticker)


def interpret_volatility_smile(ticker, overall_avg_iv_calls, overall_avg_iv_puts, avg_iv_by_strike_calls, avg_iv_by_strike_puts):
    # Interpretation based on data
    interpretation = f"**Interpretation of {ticker} Volatility Smile:**\n"

    # Average IV comparison
    interpretation += f"- The average implied volatility for call options is {overall_avg_iv_calls:.2f}%.\n"
    interpretation += f"- The average implied volatility for put options is {overall_avg_iv_puts:.2f}%.\n"

    # Volatility smile presence
    if avg_iv_by_strike_calls.var() > 0.1:  # Check if variance of IV by strike for calls is significant
        interpretation += "- The call options exhibit a noticeable 'volatility smile,' indicating varying implied volatility across different strike prices.\n"
    else:
        interpretation += "- The call options do not show a significant 'volatility smile,' suggesting more stable implied volatility across strike prices.\n"

    if avg_iv_by_strike_puts.var() > 0.1:  # Check if variance of IV by strike for puts is significant
        interpretation += "- The put options exhibit a noticeable 'volatility smile,' indicating varying implied volatility across different strike prices.\n"
    else:
        interpretation += "- The put options do not show a significant 'volatility smile,' suggesting more stable implied volatility across strike prices.\n"

    # Market sentiment based on IV
    market_sentiment = "bullish" if overall_avg_iv_calls > overall_avg_iv_puts else "bearish"
    interpretation += f"- The overall market sentiment is {market_sentiment}, inferred from the average implied volatility of calls and puts.\n"

    # Conclusion based on data trends
    if overall_avg_iv_calls > overall_avg_iv_puts:
        interpretation += "- The higher implied volatility in call options suggests that traders expect upward price movements or higher uncertainty in the stock price.\n"
    else:
        interpretation += "- The higher implied volatility in put options suggests that traders expect downward price movements or higher uncertainty in the stock price.\n"

    print(interpretation)  # Print the interpretation

# Generate interpretation based on the results from the plot function
interpret_volatility_smile(ticker, overall_avg_iv_calls, overall_avg_iv_puts, avg_iv_by_strike_calls, avg_iv_by_strike_puts)


**Interpretation of GOOG Volatility Smile:**
- The average implied volatility for call options is 39.06%.
- The average implied volatility for put options is 37.33%.
- The call options exhibit a noticeable 'volatility smile,' indicating varying implied volatility across different strike prices.
- The put options exhibit a noticeable 'volatility smile,' indicating varying implied volatility across different strike prices.
- The overall market sentiment is bullish, inferred from the average implied volatility of calls and puts.
- The higher implied volatility in call options suggests that traders expect upward price movements or higher uncertainty in the stock price.



## 3. Open Interest

This block of code defines two functions to analyze and interpret the open interest of options data for a given ticker. The plot_open_interest function filters the options data into calls and puts, and plots their open interest against strike prices using subplots for each expiration date. It also calculates the overall average open interest for both calls and puts, which is returned along with the filtered data. The interpret_open_interest function uses these results to generate insights, comparing the average open interest of calls and puts, identifying the strike prices with the highest open interest, and inferring market sentiment based on these trends.

In [ ]:
import plotly.graph_objects as go  # Import Plotly for graph objects
from plotly.subplots import make_subplots  # Import Plotly for creating subplots

def plot_open_interest(options_data, recent_price, ticker):
    if options_data is not None:  # Check if options data is available
        calls_data = options_data[options_data["type"] == "call"]  # Filter for call options
        puts_data = options_data[options_data["type"] == "put"]  # Filter for put options

        expirations = options_data['expiration'].unique()  # Get unique expiration dates
        color_map_2d = px.colors.qualitative.Prism  # Define a color map for plotting

        fig = make_subplots(rows=1, cols=2, subplot_titles=["Calls Open Interest", "Puts Open Interest"], shared_yaxes=True)  # Create subplots for calls and puts

        for exp, color in zip(expirations, color_map_2d):  # Loop through expiration dates and colors
            exp_calls = calls_data[calls_data["expiration"] == exp]  # Filter call options by expiration date
            exp_puts = puts_data[puts_data["expiration"] == exp]  # Filter put options by expiration date

            fig.add_trace(go.Scatter(x=exp_calls["strike"], y=exp_calls["openInterest"], mode='markers',
                                     marker=dict(color=color), name=exp), row=1, col=1)  # Plot call options
            fig.add_trace(go.Scatter(x=exp_puts["strike"], y=exp_puts["openInterest"], mode='markers',
                                     marker=dict(color=color), name=exp, showlegend=False), row=1, col=2)  # Plot put options

        overall_avg_oi_calls = calls_data["openInterest"].mean()  # Calculate overall average open interest for calls
        overall_avg_oi_puts = puts_data["openInterest"].mean()  # Calculate overall average open interest for puts

        fig.update_layout(title=f"{ticker} Open Interest by Strike - Current Price: {recent_price:.2f}", showlegend=True, legend_title_text='Expiration Date')  # Update layout
        fig.update_xaxes(title_text="Strike Price")  # Set x-axis title
        fig.update_yaxes(title_text="Open Interest")  # Set y-axis title

        fig.show()  # Display the plot

        # Return necessary data for interpretation
        return overall_avg_oi_calls, overall_avg_oi_puts, calls_data, puts_data

# Assuming options_data and recent_price are already defined
overall_avg_oi_calls, overall_avg_oi_puts, calls_data, puts_data = plot_open_interest(options_data, recent_price, ticker)

def interpret_open_interest(ticker, overall_avg_oi_calls, overall_avg_oi_puts, calls_data, puts_data):
    # Interpretation based on data
    interpretation = f"**Interpretation of {ticker} Open Interest:**\n"

    # Average Open Interest comparison
    interpretation += f"- The average open interest for call options is {overall_avg_oi_calls:.2f} contracts.\n"
    interpretation += f"- The average open interest for put options is {overall_avg_oi_puts:.2f} contracts.\n"

    # Open Interest trends
    if overall_avg_oi_calls > overall_avg_oi_puts:  # Compare average open interest between calls and puts
        interpretation += "- Call options have higher average open interest, indicating higher trading activity and interest in calls.\n"
    else:
        interpretation += "- Put options have higher average open interest, indicating higher trading activity and interest in puts.\n"

    # Highest Open Interest
    highest_oi_call = calls_data.loc[calls_data['openInterest'].idxmax()]  # Identify the call option with the highest open interest
    highest_oi_put = puts_data.loc[puts_data['openInterest'].idxmax()]  # Identify the put option with the highest open interest

    interpretation += f"- The strike price with the highest open interest for calls is {highest_oi_call['strike']} with {highest_oi_call['openInterest']} contracts.\n"
    interpretation += f"- The strike price with the highest open interest for puts is {highest_oi_put['strike']} with {highest_oi_put['openInterest']} contracts.\n"

    # Conclusion based on data trends
    if overall_avg_oi_calls > overall_avg_oi_puts:  # Infer market sentiment based on open interest
        interpretation += "- The higher open interest in call options suggests that traders might be anticipating upward price movements.\n"
    else:
        interpretation += "- The higher open interest in put options suggests that traders might be anticipating downward price movements.\n"

    print(interpretation)  # Print the interpretation

# Generate interpretation based on the results from the plot function
interpret_open_interest(ticker, overall_avg_oi_calls, overall_avg_oi_puts, calls_data, puts_data)


**Interpretation of GOOG Open Interest:**
- The average open interest for call options is 2298.43 contracts.
- The average open interest for put options is 2155.37 contracts.
- Call options have higher average open interest, indicating higher trading activity and interest in calls.
- The strike price with the highest open interest for calls is 175.0 with 11145 contracts.
- The strike price with the highest open interest for puts is 175.0 with 13781 contracts.
- The higher open interest in call options suggests that traders might be anticipating upward price movements.



## 4. Volume Analysis

This block of code defines two functions to analyze and interpret the trading volume of options data for a given ticker. The plot_volume function filters the options data into calls and puts, and plots their trading volumes against strike prices using subplots for each expiration date. It calculates the overall average trading volume for both calls and puts, which is returned along with the filtered data. The interpret_volume function uses these results to generate insights, comparing the average volume of calls and puts, identifying the strike prices with the highest volume, and inferring market sentiment based on these trends.

In [ ]:
import plotly.graph_objects as go  # Import Plotly for graph objects
from plotly.subplots import make_subplots  # Import Plotly for creating subplots

def plot_volume(options_data, recent_price, ticker):
    if options_data is not None:  # Check if options data is available
        calls_data = options_data[options_data["type"] == "call"]  # Filter for call options
        puts_data = options_data[options_data["type"] == "put"]  # Filter for put options

        expirations = options_data['expiration'].unique()  # Get unique expiration dates
        color_map_2d = px.colors.qualitative.Prism  # Define a color map for plotting

        fig = make_subplots(rows=1, cols=2, subplot_titles=["Calls Volume", "Puts Volume"], shared_yaxes=True)  # Create subplots for calls and puts

        for exp, color in zip(expirations, color_map_2d):  # Loop through expiration dates and colors
            exp_calls = calls_data[calls_data["expiration"] == exp]  # Filter call options by expiration date
            exp_puts = puts_data[puts_data["expiration"] == exp]  # Filter put options by expiration date

            fig.add_trace(go.Scatter(x=exp_calls["strike"], y=exp_calls["volume"], mode='markers',
                                     marker=dict(color=color), name=exp), row=1, col=1)  # Plot call options
            fig.add_trace(go.Scatter(x=exp_puts["strike"], y=exp_puts["volume"], mode='markers',
                                     marker=dict(color=color), name=exp, showlegend=False), row=1, col=2)  # Plot put options

        overall_avg_vol_calls = calls_data["volume"].mean()  # Calculate overall average volume for calls
        overall_avg_vol_puts = puts_data["volume"].mean()  # Calculate overall average volume for puts

        fig.update_layout(title=f"{ticker} Volume by Strike - Current Price: {recent_price:.2f}", showlegend=True, legend_title_text='Expiration Date')  # Update layout
        fig.update_xaxes(title_text="Strike Price")  # Set x-axis title
        fig.update_yaxes(title_text="Volume")  # Set y-axis title

        fig.show()  # Display the plot

        # Return necessary data for interpretation
        return overall_avg_vol_calls, overall_avg_vol_puts, calls_data, puts_data

# Assuming options_data and recent_price are already defined
overall_avg_vol_calls, overall_avg_vol_puts, calls_data, puts_data = plot_volume(options_data, recent_price, ticker)

def interpret_volume(ticker, overall_avg_vol_calls, overall_avg_vol_puts, calls_data, puts_data):
    # Interpretation based on data
    interpretation = f"**Interpretation of {ticker} Volume Analysis:**\n"

    # Average Volume comparison
    interpretation += f"- The average volume for call options is {overall_avg_vol_calls:.2f} contracts.\n"
    interpretation += f"- The average volume for put options is {overall_avg_vol_puts:.2f} contracts.\n"

    # Volume trends
    if overall_avg_vol_calls > overall_avg_vol_puts:  # Compare average volume between calls and puts
        interpretation += "- Call options have higher average volume, indicating higher trading activity and interest in calls.\n"
    else:
        interpretation += "- Put options have higher average volume, indicating higher trading activity and interest in puts.\n"

    # Highest Volume
    highest_vol_call = calls_data.loc[calls_data['volume'].idxmax()]  # Identify the call option with the highest volume
    highest_vol_put = puts_data.loc[puts_data['volume'].idxmax()]  # Identify the put option with the highest volume

    interpretation += f"- The strike price with the highest volume for calls is {highest_vol_call['strike']} with {highest_vol_call['volume']} contracts.\n"
    interpretation += f"- The strike price with the highest volume for puts is {highest_vol_put['strike']} with {highest_vol_put['volume']} contracts.\n"

    # Conclusion based on data trends
    if overall_avg_vol_calls > overall_avg_vol_puts:  # Infer market sentiment based on volume
        interpretation += "- The higher volume in call options suggests that traders are more actively trading calls, possibly anticipating upward price movements.\n"
    else:
        interpretation += "- The higher volume in put options suggests that traders are more actively trading puts, possibly anticipating downward price movements.\n"

    print(interpretation)  # Print the interpretation

# Generate interpretation based on the results from the plot function
interpret_volume(ticker, overall_avg_vol_calls, overall_avg_vol_puts, calls_data, puts_data)


**Interpretation of GOOG Volume Analysis:**
- The average volume for call options is 1453.70 contracts.
- The average volume for put options is 743.63 contracts.
- Call options have higher average volume, indicating higher trading activity and interest in calls.
- The strike price with the highest volume for calls is 14    167.5
14    170.0
14    155.0
Name: strike, dtype: float64 with 14    20564.0
14      952.0
14        1.0
Name: volume, dtype: float64 contracts.
- The strike price with the highest volume for puts is 17    165.0
17    152.5
17    180.0
17    170.0
Name: strike, dtype: float64 with 17    10075.0
17      452.0
17      161.0
17       45.0
Name: volume, dtype: float64 contracts.
- The higher volume in call options suggests that traders are more actively trading calls, possibly anticipating upward price movements.



## 5. Volatility Surface Plot

This block of code defines two functions to analyze and interpret the implied volatility of put and call options data for a given ticker. The main plot function filters the options data to get put/call options, then plots their implied volatility against strike prices and expiration dates in a 3D scatter plot. It returns the filtered puts data for further analysis. The interpret  function uses this data to generate insights, providing an overall summary of implied volatility, identifying the strike price with the highest implied volatility, and analyzing implied volatility trends by expiration dates and strike prices.

### Puts Volatility Surface

In [ ]:
import plotly.express as px  # Import Plotly Express for easy plotting

def plot_3d_puts_implied_volatility(options_data, ticker):
    if options_data is not None:  # Check if options data is available
        puts_data = options_data[options_data["type"] == "put"]  # Filter for put options
        expirations = options_data['expiration'].unique()  # Get unique expiration dates
        color_map_3d = {exp: color for exp, color in zip(expirations, px.colors.qualitative.Prism)}  # Define a color map for plotting

        # Create a 3D scatter plot for puts implied volatility
        fig1 = px.scatter_3d(puts_data, x='strike', y='expiration', z='implied_volatility',
                             color='expiration', color_discrete_map=color_map_3d,
                             title=f'{ticker} Put Options Implied Volatility',
                             labels={'strike': 'Strike Price', 'expiration': 'Expiration Date', 'implied_volatility': 'Implied Volatility (%)'},
                             hover_name='expiration')

        fig1.show()  # Display the plot

        # Return necessary data for interpretation
        return puts_data

# Assuming options_data and ticker are already defined
puts_data = plot_3d_puts_implied_volatility(options_data, ticker)

def interpret_3d_puts_implied_volatility(ticker, puts_data):
    # Interpretation based on data
    interpretation = f"**Interpretation of {ticker} Put Options Implied Volatility (3D Scatter Plot):**\n"

    # Overall Summary
    overall_avg_iv_puts = puts_data["implied_volatility"].mean()  # Calculate overall average implied volatility for puts
    interpretation += f"- The average implied volatility for put options is {overall_avg_iv_puts:.2f}%.\n"

    # Highest Implied Volatility
    highest_iv_put_idx = puts_data['implied_volatility'].idxmax()  # Find the index of the highest implied volatility
    highest_iv_put = puts_data.loc[highest_iv_put_idx]  # Get the row with the highest implied volatility
    if isinstance(highest_iv_put, pd.Series):  # Check if the result is a single row
        strike_price = highest_iv_put['strike']
        implied_volatility = highest_iv_put['implied_volatility']
        expiration_date = highest_iv_put['expiration']
    else:  # If multiple rows are returned, take the first one
        strike_price = highest_iv_put.iloc[0]['strike']
        implied_volatility = highest_iv_put.iloc[0]['implied_volatility']
        expiration_date = highest_iv_put.iloc[0]['expiration']

    interpretation += f"- The strike price with the highest implied volatility for puts is {strike_price} with {implied_volatility:.2f}% implied volatility, expiring on {expiration_date}.\n"

    # Analysis by Expiration Dates
    interpretation += "\n**Implied Volatility by Expiration Dates:**\n"
    for exp in puts_data['expiration'].unique():  # Loop through each expiration date
        exp_data = puts_data[puts_data['expiration'] == exp]  # Filter data for the expiration date
        avg_iv_exp = exp_data['implied_volatility'].mean()  # Calculate average IV for the expiration date
        interpretation += f"- Average IV for puts expiring on {exp}: {avg_iv_exp:.2f}%.\n"

    # Analysis by Strike Prices
    interpretation += "\n**Implied Volatility by Strike Prices:**\n"
    strike_price_bins = pd.cut(puts_data['strike'], bins=5)  # Create bins for strike prices
    grouped_strike_data = puts_data.groupby(strike_price_bins)['implied_volatility'].mean()  # Calculate average IV for each bin
    for interval, avg_iv_strike in grouped_strike_data.items():  # Loop through each bin
        interpretation += f"- Average IV for puts with strike prices in range {interval}: {avg_iv_strike:.2f}%.\n"

    # Implied Volatility Trends
    iv_variability = puts_data['implied_volatility'].std()  # Calculate the standard deviation of IV
    if iv_variability > 10:  # Check if IV variability is significant
        interpretation += "\n- There is significant variability in implied volatility across different strike prices and expiration dates, indicating diverse market expectations and uncertainty.\n"
    else:
        interpretation += "\n- The implied volatility is relatively stable across different strike prices and expiration dates, indicating consistent market expectations.\n"

    # Cohesive Story
    interpretation += "\n**Overall Analysis:**\n"
    interpretation += f"- The average IV of {overall_avg_iv_puts:.2f}% suggests a certain level of expected volatility in the underlying asset. "
    interpretation += f"The highest IV of {implied_volatility:.2f}% at the strike price of {strike_price} and expiration date of {expiration_date} indicates significant uncertainty or expected price movement around that particular strike and time.\n"
    interpretation += "- Expiration dates and strike prices both show variations in IV, suggesting that traders' expectations of volatility differ based on the specific terms of the options. "
    interpretation += "Higher IV in shorter expirations might indicate expected near-term volatility, while longer expirations with lower IV can suggest stability over time.\n"

    print(interpretation)  # Print the interpretation

# Generate interpretation based on the results from the plot function
interpret_3d_puts_implied_volatility(ticker, puts_data)

**Interpretation of GOOG Put Options Implied Volatility (3D Scatter Plot):**
- The average implied volatility for put options is 37.33%.
- The strike price with the highest implied volatility for puts is 177.5 with 106.93% implied volatility, expiring on 2025-03-14.

**Implied Volatility by Expiration Dates:**
- Average IV for puts expiring on 2025-03-14: 55.74%.
- Average IV for puts expiring on 2025-03-21: 34.48%.
- Average IV for puts expiring on 2025-03-28: 32.27%.
- Average IV for puts expiring on 2025-04-04: 31.54%.
- Average IV for puts expiring on 2025-04-11: 32.00%.
- Average IV for puts expiring on 2025-04-17: 29.16%.
- Average IV for puts expiring on 2025-04-25: 33.35%.
- Average IV for puts expiring on 2025-05-16: 31.92%.

**Implied Volatility by Strike Prices:**
- Average IV for puts with strike prices in range (152.47, 158.5]: 41.33%.
- Average IV for puts with strike prices in range (158.5, 164.5]: 32.52%.
- Average IV for puts with strike prices in range (164.5, 170.5]:

<ipython-input-6-c26639dd9fee>:56: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



### Calls Volatility Surface

In [ ]:
import plotly.express as px  # Import Plotly Express for easy plotting

def plot_3d_calls_implied_volatility(options_data, ticker):
    if options_data is not None:  # Check if options data is available
        calls_data = options_data[options_data["type"] == "call"]  # Filter for call options
        expirations = options_data['expiration'].unique()  # Get unique expiration dates
        color_map_3d = {exp: color for exp, color in zip(expirations, px.colors.qualitative.Prism)}  # Define a color map for plotting

        # Create a 3D scatter plot for calls implied volatility
        fig2 = px.scatter_3d(calls_data, x='strike', y='expiration', z='implied_volatility',
                             color='expiration', color_discrete_map=color_map_3d,
                             title=f'{ticker} Call Options Implied Volatility',
                             labels={'strike': 'Strike Price', 'expiration': 'Expiration Date', 'implied_volatility': 'Implied Volatility (%)'},
                             hover_name='expiration')

        fig2.show()  # Display the plot

        # Return necessary data for interpretation
        return calls_data

# Assuming options_data and ticker are already defined
calls_data = plot_3d_calls_implied_volatility(options_data, ticker)

def interpret_3d_calls_implied_volatility(ticker, calls_data):
    # Interpretation based on data
    interpretation = f"**Interpretation of {ticker} Call Options Implied Volatility (3D Scatter Plot):**\n"

    # Overall Summary
    overall_avg_iv_calls = calls_data["implied_volatility"].mean()  # Calculate overall average implied volatility for calls
    interpretation += f"- The average implied volatility for call options is {overall_avg_iv_calls:.2f}%.\n"

    # Highest Implied Volatility
    highest_iv_call_idx = calls_data['implied_volatility'].idxmax()  # Find the index of the highest implied volatility
    highest_iv_call = calls_data.loc[highest_iv_call_idx]  # Get the row with the highest implied volatility
    if isinstance(highest_iv_call, pd.Series):  # Check if the result is a single row
        strike_price = highest_iv_call['strike']
        implied_volatility = highest_iv_call['implied_volatility']
        expiration_date = highest_iv_call['expiration']
    else:  # If multiple rows are returned, take the first one
        strike_price = highest_iv_call.iloc[0]['strike']
        implied_volatility = highest_iv_call.iloc[0]['implied_volatility']
        expiration_date = highest_iv_call.iloc[0]['expiration']

    interpretation += f"- The strike price with the highest implied volatility for calls is {strike_price} with {implied_volatility:.2f}% implied volatility, expiring on {expiration_date}.\n"

    # Analysis by Expiration Dates
    interpretation += "\n**Implied Volatility by Expiration Dates:**\n"
    for exp in calls_data['expiration'].unique():  # Loop through each expiration date
        exp_data = calls_data[calls_data['expiration'] == exp]  # Filter data for the expiration date
        avg_iv_exp = exp_data['implied_volatility'].mean()  # Calculate average IV for the expiration date
        interpretation += f"- Average IV for calls expiring on {exp}: {avg_iv_exp:.2f}%.\n"

    # Analysis by Strike Prices
    interpretation += "\n**Implied Volatility by Strike Prices:**\n"
    strike_price_bins = pd.cut(calls_data['strike'], bins=5)  # Create bins for strike prices
    grouped_strike_data = calls_data.groupby(strike_price_bins)['implied_volatility'].mean()  # Calculate average IV for each bin
    for interval, avg_iv_strike in grouped_strike_data.items():  # Loop through each bin
        interpretation += f"- Average IV for calls with strike prices in range {interval}: {avg_iv_strike:.2f}%.\n"

    # Implied Volatility Trends
    iv_variability = calls_data['implied_volatility'].std()  # Calculate the standard deviation of IV
    if iv_variability > 10:  # Check if IV variability is significant
        interpretation += "\n- There is significant variability in implied volatility across different strike prices and expiration dates, indicating diverse market expectations and uncertainty.\n"
    else:
        interpretation += "\n- The implied volatility is relatively stable across different strike prices and expiration dates, indicating consistent market expectations.\n"

    # Cohesive Story
    interpretation += "\n**Overall Analysis:**\n"
    interpretation += f"- The average IV of {overall_avg_iv_calls:.2f}% suggests a certain level of expected volatility in the underlying asset. "
    interpretation += f"The highest IV of {implied_volatility:.2f}% at the strike price of {strike_price} and expiration date of {expiration_date} indicates significant uncertainty or expected price movement around that particular strike and time.\n"
    interpretation += "- Expiration dates and strike prices both show variations in IV, suggesting that traders' expectations of volatility differ based on the specific terms of the options. "
    interpretation += "Higher IV in shorter expirations might indicate expected near-term volatility, while longer expirations with lower IV can suggest stability over time.\n"

    print(interpretation)  # Print the interpretation

# Generate interpretation based on the results from the plot function
interpret_3d_calls_implied_volatility(ticker, calls_data)


<ipython-input-7-2f03c638b25c>:56: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



**Interpretation of GOOG Call Options Implied Volatility (3D Scatter Plot):**
- The average implied volatility for call options is 39.06%.
- The strike price with the highest implied volatility for calls is 157.5 with 96.48% implied volatility, expiring on 2025-03-14.

**Implied Volatility by Expiration Dates:**
- Average IV for calls expiring on 2025-03-14: 53.47%.
- Average IV for calls expiring on 2025-03-21: 36.28%.
- Average IV for calls expiring on 2025-03-28: 34.54%.
- Average IV for calls expiring on 2025-04-04: 34.28%.
- Average IV for calls expiring on 2025-04-11: 34.77%.
- Average IV for calls expiring on 2025-04-17: 32.73%.
- Average IV for calls expiring on 2025-04-25: 34.43%.
- Average IV for calls expiring on 2025-05-16: 36.84%.

**Implied Volatility by Strike Prices:**
- Average IV for calls with strike prices in range (152.47, 158.5]: 57.55%.
- Average IV for calls with strike prices in range (158.5, 164.5]: 42.26%.
- Average IV for calls with strike prices in range (1

## 6. Historical IV Approximation

This block of code defines functions to fetch, plot, and interpret the historical implied volatility (IV) of a given stock ticker. The get_historical_iv function downloads historical stock data and calculates the implied volatility as the percentage difference between the high and low prices. This approximation is valid because the high-low range reflects daily market sentiment and captures significant price movements indicative of volatility. The plot_historical_iv function uses this data to plot a time series of historical implied volatility, highlighting the most recent value. The interpret_historical_iv function analyzes the historical implied volatility, including averages, maximum and minimum values, yearly trends, and recent changes, to infer market expectations and volatility trends over time.

In [ ]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

def get_historical_iv(ticker, start_date, end_date):
    # Download historical stock data
    hist_data = yf.download(ticker, start=start_date, end=end_date)

    # Calculate historical implied volatility as the percentage difference between high and low prices
    hist_data['IV'] = (hist_data['High'] - hist_data['Low']) / hist_data['Low'] * 100

    return hist_data

def plot_historical_iv(ticker, start_date):
    # Set the end date to today's date
    end_date = datetime.today().strftime('%Y-%m-%d')

    # Fetch historical implied volatility data for the given ticker
    historical_iv = get_historical_iv(ticker, start_date, end_date)

    # Plot historical implied volatility using Plotly
    fig = go.Figure()

    # Add historical IV line plot
    fig.add_trace(go.Scatter(x=historical_iv.index, y=historical_iv['IV'], mode='lines', name='Historical IV'))

    # Calculate current IV (using the latest data point)
    current_iv = historical_iv['IV'].iloc[-1]

    # Add current IV as a dot on the plot
    fig.add_trace(go.Scatter(x=[historical_iv.index[-1]], y=[current_iv], mode='markers', name='Current IV',
                             marker=dict(color='red', size=10)))

    # Update plot layout
    fig.update_layout(
        title=f"{ticker} Historical Implied Volatility",
        xaxis_title="Date",
        yaxis_title="Implied Volatility (%)",
        legend_title="Legend"
    )

    # Show the plot
    fig.show()

    # Return necessary data for interpretation
    return historical_iv, current_iv



start_date = "2020-01-01"
historical_iv, current_iv = plot_historical_iv(ticker, start_date)

def interpret_historical_iv(ticker, historical_iv, current_iv):
    # Interpretation based on data
    interpretation = f"**Interpretation of {ticker} Historical Implied Volatility:**\n"

    # Overall Summary
    avg_iv = historical_iv["IV"].mean()
    max_iv = historical_iv["IV"].max()
    min_iv = historical_iv["IV"].min()

    interpretation += f"- The average implied volatility over the period is {avg_iv:.2f}%.\n"
    interpretation += f"- The maximum implied volatility recorded was {max_iv:.2f}%.\n"
    interpretation += f"- The minimum implied volatility recorded was {min_iv:.2f}%.\n"
    interpretation += f"- The current implied volatility is {current_iv:.2f}%.\n"

    # Trends over time
    historical_iv['Year'] = historical_iv.index.year
    avg_iv_by_year = historical_iv.groupby('Year')['IV'].mean()
    interpretation += "\n**Average Implied Volatility by Year:**\n"
    for year, avg_iv in avg_iv_by_year.items():
        interpretation += f"- {year}: {avg_iv:.2f}%.\n"

    # Variability Analysis
    iv_variability = historical_iv['IV'].std()
    interpretation += f"\n- The standard deviation of implied volatility over the period is {iv_variability:.2f}, indicating {'high' if iv_variability > 10 else 'low'} variability in implied volatility.\n"

    # Recent Trends
    recent_trend = "increased" if historical_iv['IV'].iloc[-1] > historical_iv['IV'].iloc[-30] else "decreased"
    interpretation += f"\n- In the last 30 days, the implied volatility has {recent_trend} compared to the previous period.\n"

    print(interpretation)

# Generate interpretation based on the results from the plot function
interpret_historical_iv(ticker, historical_iv, current_iv)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


**Interpretation of GOOG Historical Implied Volatility:**
- The average implied volatility over the period is 2.31%.
- The maximum implied volatility recorded was 9.23%.
- The minimum implied volatility recorded was 0.45%.
- The current implied volatility is 2.47%.

**Average Implied Volatility by Year:**
- 2020: 2.63%.
- 2021: 1.81%.
- 2022: 2.88%.
- 2023: 2.25%.
- 2024: 2.00%.
- 2025: 2.33%.

- The standard deviation of implied volatility over the period is 1.15, indicating low variability in implied volatility.

- In the last 30 days, the implied volatility has increased compared to the previous period.



## 7. Put to Call Ratio

 The calculate_put_call_ratio function separates the options data into calls and puts, sums their volumes, and calculates the Put/Call ratio. This ratio is a key indicator of market sentiment, where a higher ratio suggests bearish sentiment and a lower ratio suggests bullish sentiment. The interpret_put_call_ratio function provides a detailed interpretation of the calculated ratio, including total volumes of puts and calls and the inferred market sentiment based on the ratio.

In [ ]:
def calculate_put_call_ratio(options_data):
    if options_data is not None:  # Check if options data is available
        calls_data = options_data[options_data["type"] == "call"]  # Filter for call options
        puts_data = options_data[options_data["type"] == "put"]  # Filter for put options

        total_puts = puts_data['volume'].sum()  # Sum the volume of put options
        total_calls = calls_data['volume'].sum()  # Sum the volume of call options
        put_call_ratio = total_puts / total_calls  # Calculate the Put/Call ratio

        print(f"Put/Call Ratio: {put_call_ratio:.2f}")  # Print the Put/Call ratio

        # Return necessary data for interpretation
        return total_puts, total_calls, put_call_ratio

# Assuming options_data is already defined
total_puts, total_calls, put_call_ratio = calculate_put_call_ratio(options_data)

def interpret_put_call_ratio(ticker, total_puts, total_calls, put_call_ratio):
    # Interpretation based on data
    interpretation = f"**Interpretation of {ticker} Put/Call Ratio:**\n"

    # Put/Call Ratio
    interpretation += f"- The Put/Call Ratio is {put_call_ratio:.2f}.\n"

    # Total Puts and Calls
    interpretation += f"- Total volume of puts: {total_puts}\n"
    interpretation += f"- Total volume of calls: {total_calls}\n"

    # Market Sentiment
    if put_call_ratio > 1:  # Check if the Put/Call ratio indicates bearish sentiment
        interpretation += "- A Put/Call Ratio greater than 1 indicates a bearish sentiment in the market, as more puts are being traded relative to calls.\n"
    elif put_call_ratio < 1:  # Check if the Put/Call ratio indicates bullish sentiment
        interpretation += "- A Put/Call Ratio less than 1 indicates a bullish sentiment in the market, as more calls are being traded relative to puts.\n"
    else:  # If the Put/Call ratio is exactly 1
        interpretation += "- A Put/Call Ratio of 1 indicates a neutral sentiment in the market, with equal volumes of puts and calls being traded.\n"

    print(interpretation)  # Print the interpretation

# Generate interpretation based on the results from the calculation function
interpret_put_call_ratio(ticker, total_puts, total_calls, put_call_ratio)


Put/Call Ratio: 0.53
**Interpretation of GOOG Put/Call Ratio:**
- The Put/Call Ratio is 0.53.
- Total volume of puts: 46105.0
- Total volume of calls: 87222.0
- A Put/Call Ratio less than 1 indicates a bullish sentiment in the market, as more calls are being traded relative to puts.



## 8. Greeks Analysis

The calculate_greeks function computes the Greeks (Delta, Gamma, Theta, Vega) for given options using the Black-Scholes model. The plot_greeks function uses this to add the Greeks to options data and plots them against strike prices in a multi-subplot graph. The interpret_greeks function provides an analysis of the calculated Greeks, including averages and their implications for market sentiment.

In [ ]:
import numpy as np  # Import NumPy for numerical operations
from py_vollib.black_scholes.greeks import analytical  # Import the analytical Greeks calculations from py_vollib
from plotly.subplots import make_subplots  # Import Plotly for creating subplots
import plotly.graph_objects as go  # Import Plotly for graph objects

def calculate_greeks(option_type, S, K, T, r, sigma):
    greeks = {}
    try:
        # Set flag based on option type
        flag = "c" if option_type == "call" else "p"

        # Calculate the Greeks using the analytical methods
        greeks['delta'] = analytical.delta(flag, S, K, T, r, sigma)
        greeks['gamma'] = analytical.gamma(flag, S, K, T, r, sigma)
        greeks['theta'] = analytical.theta(flag, S, K, T, r, sigma)
        greeks['vega'] = analytical.vega(flag, S, K, T, r, sigma)

    except Exception as e:
        # Assign NaN if any error occurs
        greeks = {'delta': np.nan, 'gamma': np.nan, 'theta': np.nan, 'vega': np.nan}

    return greeks  # Return the calculated Greeks

def plot_greeks(options_data, recent_price, ticker):
    if options_data is not None:  # Check if options data is available
        r = 0.01  # Assuming a risk-free rate of 1%
        calls_data = options_data[options_data["type"] == "call"].copy()  # Filter and copy call options data
        puts_data = options_data[options_data["type"] == "put"].copy()  # Filter and copy put options data

        for index, row in calls_data.iterrows():  # Iterate through each call option
            T = (pd.to_datetime(row['expiration']) - pd.Timestamp.now()).days / 365.25  # Calculate time to expiration
            if T > 0 and row['impliedVolatility'] > 0:  # Ensure positive time to expiration and positive volatility
                greeks = calculate_greeks("call", recent_price, row['strike'], T, r, row['impliedVolatility'])
            else:
                greeks = {'delta': np.nan, 'gamma': np.nan, 'theta': np.nan, 'vega': np.nan}
            for greek, value in greeks.items():
                calls_data.at[index, greek] = value  # Add Greeks to calls data

        for index, row in puts_data.iterrows():  # Iterate through each put option
            T = (pd.to_datetime(row['expiration']) - pd.Timestamp.now()).days / 365.25  # Calculate time to expiration
            if T > 0 and row['impliedVolatility'] > 0:  # Ensure positive time to expiration and positive volatility
                greeks = calculate_greeks("put", recent_price, row['strike'], T, r, row['impliedVolatility'])
            else:
                greeks = {'delta': np.nan, 'gamma': np.nan, 'theta': np.nan, 'vega': np.nan}
            for greek, value in greeks.items():
                puts_data.at[index, greek] = value  # Add Greeks to puts data

        # Create subplots for Delta, Gamma, Theta, and Vega
        fig = make_subplots(rows=2, cols=2, subplot_titles=["Delta", "Gamma", "Theta", "Vega"], shared_yaxes=True)

        # Plot Delta for calls and puts
        fig.add_trace(go.Scatter(x=calls_data["strike"], y=calls_data["delta"], mode='markers', marker=dict(color='blue'), name="Delta (Calls)"), row=1, col=1)
        fig.add_trace(go.Scatter(x=puts_data["strike"], y=puts_data["delta"], mode='markers', marker=dict(color='red'), name="Delta (Puts)"), row=1, col=1)

        # Plot Gamma for calls and puts
        fig.add_trace(go.Scatter(x=calls_data["strike"], y=calls_data["gamma"], mode='markers', marker=dict(color='blue'), name="Gamma (Calls)"), row=1, col=2)
        fig.add_trace(go.Scatter(x=puts_data["strike"], y=puts_data["gamma"], mode='markers', marker=dict(color='red'), name="Gamma (Puts)"), row=1, col=2)

        # Plot Theta for calls and puts
        fig.add_trace(go.Scatter(x=calls_data["strike"], y=calls_data["theta"], mode='markers', marker=dict(color='blue'), name="Theta (Calls)"), row=2, col=1)
        fig.add_trace(go.Scatter(x=puts_data["strike"], y=puts_data["theta"], mode='markers', marker=dict(color='red'), name="Theta (Puts)"), row=2, col=1)

        # Plot Vega for calls and puts
        fig.add_trace(go.Scatter(x=calls_data["strike"], y=calls_data["vega"], mode='markers', marker=dict(color='blue'), name="Vega (Calls)"), row=2, col=2)
        fig.add_trace(go.Scatter(x=puts_data["strike"], y=puts_data["vega"], mode='markers', marker=dict(color='red'), name="Vega (Puts)"), row=2, col=2)

        # Update layout with title and axis labels
        fig.update_layout(title=f"{ticker} Options Greeks by Strike - Current Price: {recent_price:.2f}", showlegend=True, legend_title_text='Options Type')
        fig.update_xaxes(title_text="Strike Price")
        fig.update_yaxes(title_text="Greeks Value")

        fig.show()  # Display the plot

        # Return necessary data for interpretation
        return calls_data, puts_data

# Assuming options_data and recent_price are already defined
calls_data, puts_data = plot_greeks(options_data, recent_price, ticker)

def interpret_greeks(ticker, calls_data, puts_data):
    # Interpretation based on data
    interpretation = f"**Interpretation of {ticker} Options Greeks:**\n"

    # Average Greeks for Calls
    avg_delta_calls = calls_data['delta'].mean()
    avg_gamma_calls = calls_data['gamma'].mean()
    avg_theta_calls = calls_data['theta'].mean()
    avg_vega_calls = calls_data['vega'].mean()

    # Average Greeks for Puts
    avg_delta_puts = puts_data['delta'].mean()
    avg_gamma_puts = puts_data['gamma'].mean()
    avg_theta_puts = puts_data['theta'].mean()
    avg_vega_puts = puts_data['vega'].mean()

    # Delta Interpretation
    interpretation += f"- **Delta Interpretation:**\n"
    interpretation += f"  - Calls: The average delta for calls is {avg_delta_calls:.2f}. This indicates that on average, the call options' price changes by {avg_delta_calls:.2f} for a $1 change in the underlying stock price.\n"
    interpretation += f"  - Puts: The average delta for puts is {avg_delta_puts:.2f}. This indicates that on average, the put options' price changes by {avg_delta_puts:.2f} for a $1 change in the underlying stock price.\n\n"

    # Gamma Interpretation
    interpretation += f"- **Gamma Interpretation:**\n"
    interpretation += f"  - Calls: The average gamma for calls is {avg_gamma_calls:.2f}. This suggests that the delta of call options changes by {avg_gamma_calls:.2f} for a $1 change in the underlying stock price.\n"
    interpretation += f"  - Puts: The average gamma for puts is {avg_gamma_puts:.2f}. This suggests that the delta of put options changes by {avg_gamma_puts:.2f} for a $1 change in the underlying stock price.\n\n"

    # Theta Interpretation
    interpretation += f"- **Theta Interpretation:**\n"
    interpretation += f"  - Calls: The average theta for calls is {avg_theta_calls:.2f}. This indicates that on average, the price of call options decreases by {avg_theta_calls:.2f} per day as time passes.\n"
    interpretation += f"  - Puts: The average theta for puts is {avg_theta_puts:.2f}. This indicates that on average, the price of put options decreases by {avg_theta_puts:.2f} per day as time passes.\n\n"

    # Vega Interpretation
    interpretation += f"- **Vega Interpretation:**\n"
    interpretation += f"  - Calls: The average vega for calls is {avg_vega_calls:.2f}. This suggests that the price of call options changes by {avg_vega_calls:.2f} for a 1% change in the volatility of the underlying stock.\n"
    interpretation += f"  - Puts: The average vega for puts is {avg_vega_puts:.2f}. This suggests that the price of put options changes by {avg_vega_puts:.2f} for a 1% change in the volatility of the underlying stock.\n\n"

    # Summary
    interpretation += "\n**Summary and Market Sentiment:**\n"
    if avg_delta_calls > avg_delta_puts:
        interpretation += "- The higher average delta for calls compared to puts indicates a bullish sentiment, as call options are more sensitive to upward movements in the stock price.\n"
    else:
        interpretation += "- The higher average delta for puts compared to calls indicates a bearish sentiment, as put options are more sensitive to downward movements in the stock price.\n"

    if avg_gamma_calls > avg_gamma_puts:
        interpretation += "- The higher average gamma for calls suggests that the sensitivity of call options to the underlying stock price is higher, indicating potential for larger price swings.\n"
    else:
        interpretation += "- The higher average gamma for puts suggests that the sensitivity of put options to the underlying stock price is higher, indicating potential for larger price swings.\n"

    if avg_theta_calls < avg_theta_puts:
        interpretation += "- The more negative average theta for calls indicates that call options lose value faster over time compared to puts, which may reflect higher time decay for bullish positions.\n"
    else:
        interpretation += "- The more negative average theta for puts indicates that put options lose value faster over time compared to calls, which may reflect higher time decay for bearish positions.\n"

    if avg_vega_calls > avg_vega_puts:
        interpretation += "- The higher average vega for calls suggests that call options are more sensitive to changes in volatility, which can be beneficial in volatile markets for bullish strategies.\n"
    else:
        interpretation += "- The higher average vega for puts suggests that put options are more sensitive to changes in volatility, which can be beneficial in volatile markets for bearish strategies.\n"

    print(interpretation)  # Print the interpretation

# Generate interpretation based on the results from the plot function
interpret_greeks(ticker, calls_data, puts_data)


**Interpretation of GOOG Options Greeks:**
- **Delta Interpretation:**
  - Calls: The average delta for calls is 0.45. This indicates that on average, the call options' price changes by 0.45 for a $1 change in the underlying stock price.
  - Puts: The average delta for puts is -0.46. This indicates that on average, the put options' price changes by -0.46 for a $1 change in the underlying stock price.

- **Gamma Interpretation:**
  - Calls: The average gamma for calls is 0.02. This suggests that the delta of call options changes by 0.02 for a $1 change in the underlying stock price.
  - Puts: The average gamma for puts is 0.02. This suggests that the delta of put options changes by 0.02 for a $1 change in the underlying stock price.

- **Theta Interpretation:**
  - Calls: The average theta for calls is -0.10. This indicates that on average, the price of call options decreases by -0.10 per day as time passes.
  - Puts: The average theta for puts is -0.10. This indicates that on average, 

## 9. Sentiment Score

The calculate_sentiment_score function computes the sentiment score by combining the difference between the highest implied volatility of calls and the lowest implied volatility of puts with the normalized difference between the total volume of calls and puts. The function then returns the sentiment score and a description of whether the sentiment is bullish or bearish. The interpret_sentiment_score function provides a detailed analysis of the sentiment score, explaining its components and the inferred market sentiment based on the score.

In [ ]:
def calculate_sentiment_score(options_data, total_calls, total_puts):
    if options_data is not None:  # Check if options data is available
        calls_data = options_data[options_data["type"] == "call"]
        puts_data = options_data[options_data["type"] == "put"]

        # Calculate high IV calls and low IV puts (replace with actual calculations)
        high_iv_calls = calls_data['impliedVolatility'].max()  # Example calculation
        low_iv_puts = puts_data['impliedVolatility'].min()  # Example calculation

        # Calculate sentiment score using the formula
        sentiment_score = (high_iv_calls - low_iv_puts) + (total_calls - total_puts) / (total_calls + total_puts)
        # Determine sentiment description based on the sentiment score
        sentiment_description = "Bullish" if sentiment_score > 0 else "Bearish"

        print(f"Sentiment Score: {sentiment_score:.2f} ({sentiment_description})")  # Print the sentiment score

        # Return necessary data for interpretation
        return sentiment_score, sentiment_description, high_iv_calls, low_iv_puts

# Assuming options_data, total_calls, and total_puts are already defined
sentiment_score, sentiment_description, high_iv_calls, low_iv_puts = calculate_sentiment_score(options_data, total_calls, total_puts)

def interpret_sentiment_score(ticker, sentiment_score, sentiment_description, high_iv_calls, low_iv_puts, total_calls, total_puts):
    # Interpretation based on data
    interpretation = f"**Interpretation of {ticker} Sentiment Score:**\n"

    # Sentiment Score
    interpretation += f"- The sentiment score is {sentiment_score:.2f}, indicating a {sentiment_description} market sentiment.\n"

    # Contribution to Sentiment Score
    iv_diff = high_iv_calls - low_iv_puts  # Calculate the difference in implied volatility
    volume_diff = (total_calls - total_puts) / (total_calls + total_puts)  # Calculate the normalized volume difference

    interpretation += f"- Contribution from Implied Volatility difference (high IV of calls - low IV of puts): {iv_diff:.2f}\n"
    interpretation += f"- Contribution from Volume difference (total calls - total puts) / (total calls + total puts): {volume_diff:.2f}\n"

    # Market Sentiment Analysis
    if sentiment_description == "Bullish":  # Check if the sentiment is bullish
        interpretation += "- The bullish sentiment is driven by higher implied volatility in calls compared to puts, indicating expectations of upward price movements. Additionally, a higher volume of calls compared to puts suggests more traders are positioning for a rise in the stock price.\n"
    else:  # If the sentiment is bearish
        interpretation += "- The bearish sentiment is driven by higher implied volatility in puts compared to calls, indicating expectations of downward price movements. Additionally, a higher volume of puts compared to calls suggests more traders are positioning for a decline in the stock price.\n"

    print(interpretation)  # Print the interpretation

# Generate interpretation based on the results from the calculation function
interpret_sentiment_score(ticker, sentiment_score, sentiment_description, high_iv_calls, low_iv_puts, total_calls, total_puts)

Sentiment Score: 1.17 (Bullish)
**Interpretation of GOOG Sentiment Score:**
- The sentiment score is 1.17, indicating a Bullish market sentiment.
- Contribution from Implied Volatility difference (high IV of calls - low IV of puts): 0.86
- Contribution from Volume difference (total calls - total puts) / (total calls + total puts): 0.31
- The bullish sentiment is driven by higher implied volatility in calls compared to puts, indicating expectations of upward price movements. Additionally, a higher volume of calls compared to puts suggests more traders are positioning for a rise in the stock price.



## 10. Overall Interpretation of the Results

In [ ]:
def overall_interpretation(ticker,
                           overall_avg_iv_calls, overall_avg_iv_puts, avg_iv_by_strike_calls, avg_iv_by_strike_puts,
                           overall_avg_vol_calls, overall_avg_vol_puts, calls_data, puts_data,
                           historical_iv, current_iv,
                           put_call_ratio,
                           sentiment_score, sentiment_description, high_iv_calls, low_iv_puts, total_calls, total_puts,
                           avg_delta_calls, avg_delta_puts, avg_gamma_calls, avg_gamma_puts, avg_theta_calls, avg_theta_puts, avg_vega_calls, avg_vega_puts):
    interpretation = f"**Overall Interpretation of {ticker} Market Sentiment and Conditions:**\n\n"

    # Volatility Smile Interpretation
    interpretation += f"**Volatility Smile Analysis:**\n"
    interpretation += f"- The average implied volatility for call options is {overall_avg_iv_calls:.2f}%.\n"
    interpretation += f"- The average implied volatility for put options is {overall_avg_iv_puts:.2f}%.\n"
    interpretation += f"- The market shows {'a significant' if avg_iv_by_strike_calls.var() > 0.1 else 'a stable'} volatility smile for call options.\n"
    interpretation += f"- The market shows {'a significant' if avg_iv_by_strike_puts.var() > 0.1 else 'a stable'} volatility smile for put options.\n"
    interpretation += f"- The overall market sentiment from volatility is {'bullish' if overall_avg_iv_calls > overall_avg_iv_puts else 'bearish'}.\n\n"

    # Volume Analysis Interpretation
    interpretation += f"**Volume Analysis:**\n"
    interpretation += f"- The average volume for call options is {overall_avg_vol_calls:.2f} contracts.\n"
    interpretation += f"- The average volume for put options is {overall_avg_vol_puts:.2f} contracts.\n"
    interpretation += f"- The higher volume in {'call' if overall_avg_vol_calls > overall_avg_vol_puts else 'put'} options indicates higher trading activity and interest.\n"
    highest_vol_call = calls_data.loc[calls_data['volume'].idxmax()]
    highest_vol_put = puts_data.loc[puts_data['volume'].idxmax()]
    interpretation += f"- The highest volume for calls is at strike {highest_vol_call['strike']} with {highest_vol_call['volume']} contracts.\n"
    interpretation += f"- The highest volume for puts is at strike {highest_vol_put['strike']} with {highest_vol_put['volume']} contracts.\n\n"

    # Historical IV Interpretation
    avg_iv = historical_iv["IV"].mean()
    max_iv = historical_iv["IV"].max()
    min_iv = historical_iv["IV"].min()
    interpretation += f"**Historical Implied Volatility Analysis:**\n"
    interpretation += f"- The average implied volatility over the period is {avg_iv:.2f}%.\n"
    interpretation += f"- The maximum implied volatility recorded was {max_iv:.2f}%.\n"
    interpretation += f"- The minimum implied volatility recorded was {min_iv:.2f}%.\n"
    interpretation += f"- The current implied volatility is {current_iv:.2f}%.\n"
    recent_trend = "increased" if historical_iv['IV'].iloc[-1] > historical_iv['IV'].iloc[-30] else "decreased"
    interpretation += f"- In the last 30 days, the implied volatility has {recent_trend} compared to the previous period.\n\n"

    # Put/Call Ratio Interpretation
    interpretation += f"**Put/Call Ratio Analysis:**\n"
    interpretation += f"- The Put/Call Ratio is {put_call_ratio:.2f}.\n"
    interpretation += f"- Total volume of puts: {total_puts}\n"
    interpretation += f"- Total volume of calls: {total_calls}\n"
    if put_call_ratio > 1:
        interpretation += "- A Put/Call Ratio greater than 1 indicates a bearish sentiment, with more puts being traded relative to calls.\n\n"
    elif put_call_ratio < 1:
        interpretation += "- A Put/Call Ratio less than 1 indicates a bullish sentiment, with more calls being traded relative to puts.\n\n"
    else:
        interpretation += "- A Put/Call Ratio of 1 indicates a neutral sentiment, with equal volumes of puts and calls being traded.\n\n"

    # Greeks Analysis Interpretation
    interpretation += f"**Options Greeks Analysis:**\n"
    interpretation += f"- **Delta:**\n"
    interpretation += f"  - Calls: The average delta for calls is {avg_delta_calls:.2f}.\n"
    interpretation += f"  - Puts: The average delta for puts is {avg_delta_puts:.2f}.\n"
    interpretation += f"- **Gamma:**\n"
    interpretation += f"  - Calls: The average gamma for calls is {avg_gamma_calls:.2f}.\n"
    interpretation += f"  - Puts: The average gamma for puts is {avg_gamma_puts:.2f}.\n"
    interpretation += f"- **Theta:**\n"
    interpretation += f"  - Calls: The average theta for calls is {avg_theta_calls:.2f}.\n"
    interpretation += f"  - Puts: The average theta for puts is {avg_theta_puts:.2f}.\n"
    interpretation += f"- **Vega:**\n"
    interpretation += f"  - Calls: The average vega for calls is {avg_vega_calls:.2f}.\n"
    interpretation += f"  - Puts: The average vega for puts is {avg_vega_puts:.2f}.\n"

    if avg_delta_calls > avg_delta_puts:
        interpretation += "- The higher average delta for calls suggests a bullish sentiment as calls are more sensitive to upward movements.\n"
    else:
        interpretation += "- The higher average delta for puts suggests a bearish sentiment as puts are more sensitive to downward movements.\n"
    if avg_gamma_calls > avg_gamma_puts:
        interpretation += "- The higher average gamma for calls indicates higher sensitivity of delta to underlying price changes, suggesting larger price swings for calls.\n"
    else:
        interpretation += "- The higher average gamma for puts indicates higher sensitivity of delta to underlying price changes, suggesting larger price swings for puts.\n"
    if avg_theta_calls < avg_theta_puts:
        interpretation += "- The more negative average theta for calls indicates higher time decay for bullish positions.\n"
    else:
        interpretation += "- The more negative average theta for puts indicates higher time decay for bearish positions.\n"
    if avg_vega_calls > avg_vega_puts:
        interpretation += "- The higher average vega for calls suggests calls are more sensitive to volatility changes, beneficial in volatile markets for bullish strategies.\n"
    else:
        interpretation += "- The higher average vega for puts suggests puts are more sensitive to volatility changes, beneficial in volatile markets for bearish strategies.\n\n"

    # Sentiment Score Interpretation
    interpretation += f"**Sentiment Score Analysis:**\n"
    interpretation += f"- The sentiment score is {sentiment_score:.2f}, indicating a {sentiment_description} market sentiment.\n"
    interpretation += f"- Contribution from Implied Volatility difference (high IV of calls - low IV of puts): {high_iv_calls - low_iv_puts:.2f}\n"
    interpretation += f"- Contribution from Volume difference (total calls - total puts) / (total calls + total puts): {(total_calls - total_puts) / (total_calls + total_puts):.2f}\n"
    if sentiment_description == "Bullish":
        interpretation += "- The bullish sentiment is driven by higher implied volatility in calls and higher call volumes, indicating expectations of upward price movements.\n"
    else:
        interpretation += "- The bearish sentiment is driven by higher implied volatility in puts and higher put volumes, indicating expectations of downward price movements.\n"

    print(interpretation)

# Calculate average Greeks for both calls and puts
avg_delta_calls = calls_data['delta'].mean()
avg_gamma_calls = calls_data['gamma'].mean()
avg_theta_calls = calls_data['theta'].mean()
avg_vega_calls = calls_data['vega'].mean()

avg_delta_puts = puts_data['delta'].mean()
avg_gamma_puts = puts_data['gamma'].mean()
avg_theta_puts = puts_data['theta'].mean()
avg_vega_puts = puts_data['vega'].mean()

# Generate overall interpretation based on all the results
overall_interpretation(ticker,
                       overall_avg_iv_calls, overall_avg_iv_puts, avg_iv_by_strike_calls, avg_iv_by_strike_puts,
                       overall_avg_vol_calls, overall_avg_vol_puts, calls_data, puts_data,
                       historical_iv, current_iv,
                       put_call_ratio,
                       sentiment_score, sentiment_description, high_iv_calls, low_iv_puts, total_calls, total_puts,
                       avg_delta_calls, avg_delta_puts, avg_gamma_calls, avg_gamma_puts, avg_theta_calls, avg_theta_puts, avg_vega_calls, avg_vega_puts)


**Overall Interpretation of GOOG Market Sentiment and Conditions:**

**Volatility Smile Analysis:**
- The average implied volatility for call options is 39.06%.
- The average implied volatility for put options is 37.33%.
- The market shows a significant volatility smile for call options.
- The market shows a significant volatility smile for put options.
- The overall market sentiment from volatility is bullish.

**Volume Analysis:**
- The average volume for call options is 1453.70 contracts.
- The average volume for put options is 743.63 contracts.
- The higher volume in call options indicates higher trading activity and interest.
- The highest volume for calls is at strike 14    167.5
14    170.0
14    155.0
Name: strike, dtype: float64 with 14    20564.0
14      952.0
14        1.0
Name: volume, dtype: float64 contracts.
- The highest volume for puts is at strike 17    165.0
17    152.5
17    180.0
17    170.0
Name: strike, dtype: float64 with 17    10075.0
17      452.0
17      161.